In [10]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Resultados do Active Learning

In [11]:
df1 = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df1 = df1[df1['y'].notnull()]
df1.shape

(498, 16)

In [12]:
df2 = pd.read_csv("active_labels1_done.csv", index_col=0)
df2 = df2[df2['y'].notnull()]
df2['novo'] = 1
df2.shape

(100, 18)

In [13]:
df2.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p,novo
505,Platform Overview - Machine Learning,0,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502,1


In [14]:
from sklearn.metrics import roc_auc_score, average_precision_score
average_precision_score(df2['y'],df2['p']), roc_auc_score(df2['y'],df2['p'])

(0.2037344613689981, 0.5386250885896527)

In [15]:
df = pd.concat([df1, df2.drop("p", axis=1)])

C:\Users\diego.mayer\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [16]:
df.head()

,channel_link_0,content_watch-info-tag-list,novo,og:description,og:image,og:image:height,og:image:width,og:video:height,og:video:tag,og:video:width,watch-time-text,watch-title,watch-view-count,watch7-headline,watch7-user-header,watch8-sentiment-actions,y
0,/channel/UCsvqVGtbbyHaMoevxPAq9Fg,Educação,NaN,"This video on ""How to become a Machine Learnin...",https://i.ytimg.com/vi/-5hEYRt8JE0/maxresdefau...,720.0,1280.0,720.0,simplilearn,1280.0,Publicado em 3 de set. de 2018,How to Become A Machine Learning Engineer | Ho...,28.028 visualizações,#MachineLearningAlgorithms #Datasciencecourse ...,Simplilearn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,28.028 visualizações\n\n\n\n\n\n\n\n601\n\nGos...,0.0
1,/channel/UCTPin8TK-KRSI9zo9FoxG0g,Pessoas e blogs,NaN,#FAIL :) Have fun - and join our teams: https:...,https://i.ytimg.com/vi/-7GiiT0yEyk/maxresdefau...,720.0,1280.0,720.0,employer branding,1280.0,Publicado em 16 de nov. de 2018,BLOOPERS - Behind The Scenes. | DATA SCIENCE x...,1.131 visualizações,#FAIL #insidezalando\n\n\n\n BLOOPERS - Beh...,Inside Zalando\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarr...,1.131 visualizações\n\n\n\n\n\n\n\n20\n\nGosto...,0.0
2,/channel/UC8FZ6dzFVkCACLH9YoMNFog,Licença de atribuição Creative Commons (reutil...,NaN,2019 Purdue Engineering Distinguished Lecture ...,https://i.ytimg.com/vi/-8yYFdV5SOc/maxresdefau...,720.0,1280.0,720.0,electrical engineer,1280.0,Publicado em 2 de mai. de 2019,Michael I. Jordan: Machine Learning: Dynamical...,1.816 visualizações,#purdue #michaelijordan #engineering\n\n\n\n ...,Purdue Engineering\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1.816 visualizações\n\n\n\n\n\n\n\n42\n\nGosto...,1.0
3,/channel/UC_215Y7rOAsqnFkO_hnpdIg,Ciência e tecnologia,NaN,A collection of the 5 best deep learning tools...,https://i.ytimg.com/vi/-9LLrwW1Vdo/maxresdefau...,720.0,1280.0,720.0,Watson,1280.0,Publicado em 13 de ago. de 2019,Best Deep Learning Tools - Welcome.AI,1.171 visualizações,Best Deep Learning Tools - Welcome.AI,Welcome.AI\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,1.171 visualizações\n\n\n\n\n\n\n\n14\n\nGosto...,0.0
4,/channel/UCSNeZleDn9c74yQc-EKnVTA,Ciência e tecnologia,NaN,Join Kaggle data scientist Rachael live as she...,https://i.ytimg.com/vi/-9U84J178OQ/maxresdefau...,720.0,1280.0,720.0,CS,1280.0,Transmitido ao vivo em 30 de nov. de 2018,Kaggle Live-Coding: RNNs for Sarcasm Detection...,1.228 visualizações,Kaggle Live-Coding: RNNs for Sarcasm Detection...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.228 visualizações\n\n\n\n\n\n\n\n28\n\nGosto...,1.0


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [18]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']
df_limpo['novo'] = df['novo'].fillna(0)

## 1. Limpeza da data

In [19]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [20]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [21]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [22]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [23]:
features.head()

,views,views_por_dia
0,28028,61.464912
1,1131,2.960733
2,1816,8.446512
3,1171,10.455357
4,1228,3.336957


# Aumenta validação

In [24]:
mask_train = (df_limpo['date'] < "2019-04-01") & (df_limpo['novo'] == 0)

mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (316, 2), (228,), (316,))

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [26]:
title_bow_train.shape

(228, 193)

In [27]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [28]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((228, 195), (316, 195))

In [29]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=6, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [30]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [31]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [32]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.1872802752830275, 0.5935436218282933)

ANTES  
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2  


# Aumenta treino

In [33]:
mask_train = (df_limpo['date'] < "2019-04-01")
#mask_val = (df_limpo['date'] >= "2019-04-01") & (df_limpo['novo'] == 0)
mask_val = (df_limpo['date'] >= "2019-04-01")

In [34]:
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])


mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

p = mdl.predict_proba(Xval_wtitle)[:, 1]

average_precision_score(yval, p), roc_auc_score(yval, p)

(0.19342838733336654, 0.6095324991310394)

ANTES  
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2  

Valid nova, treino antigo  
ap 0.1872802752830275, auc 0.5935436218282933

Valid antiga, treino novo  
ap 0.17697903398635972, auc 0.5964133738601823

Tudo novo  
ap 0.19342838733336654, auc 0.6095324991310394